# Robot Motion Planning

I coded these programs based on _Udacity's AI for robotics_ course videos and all the screenshots in this tutorial are taken from that course videos. 

## Function Specifications

Goal is to find a shortest path through a maze/grid in which there are hurdles as well. Grid/Maze format:
- 0 = Navigable space
- 1 = Occupied space

Define a function, search() that returns a list in the form of [optimal path length, row, col]. For the grid shown below, function should output [11, 4, 5].

If there is no valid path from the start point to the goal, function should return the string 'fail'.

![visualization of maze](visualization.png)

In [1]:
#function to calculate next cell after specificed move has been made
def get_next_cell(current_cell, move):
    next_cell = [current_cell[0] + move[0], current_cell[1] + move[1]]
    return next_cell

In [2]:
#Function to check if making the passed move is possible and valid
def is_move_valid(grid, current_cell, move):
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #make sure we are not overshooting the maze/grid boundary
    if next_cell[0] >= len(grid) or next_cell[1] >= len(grid[0]):
        return False
    
    #make sure we are not undershooting the maze/grid boundary
    if next_cell[0] < 0 or next_cell[1] < 0:
        return False
    
    #make sure the cell is not already occupied, either because 
    #it is an obstacle or we traveresed it before
    if grid[next_cell[0]][next_cell[1]] == 1:
        return False
    
    return True

In [3]:
#function to make a move, it assumes that the move
#passed is a valid move to make
def make_move(grid, current_cell, move):
    #calculate the next cell
    #calculate the next cell
    next_cell = get_next_cell(current_cell, move)
    
    #update the grid cell as marked
    grid[next_cell[0]][next_cell[1]] = 1

In [4]:
#function to find possible adjacent cells which are not occupied
def find_available_cells(grid, current_cell, moves):
    available_cells = []
    
    for move in moves:
        if (is_move_valid(grid, current_cell, move)):
            available_cells.append(get_next_cell(current_cell, move))
            
    return available_cells

In [5]:
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, function should return the string
# 'fail'
# ----------

# Grid/Maze format:
#   0 = Navigable space
#   1 = Occupied space

# grid = [[0, 0, 1, 0, 0, 0],
#         [0, 0, 1, 0, 0, 0],
#         [0, 0, 0, 0, 1, 0],
#         [0, 0, 1, 1, 1, 0],
#         [0, 0, 0, 0, 1, 0]]

# grid = [[0, 1],
#         [0, 0]]

# grid = [[0, 1, 1, 1, 1],
#         [0, 1, 0, 0, 0],
#         [0, 0, 0, 1, 0],
#         [1, 1, 1, 1, 0],
#         [0, 0, 0, 1, 0]]

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

In [6]:
def add_cells_to_open_list(open_cells, cells, new_cost):
    for cell in cells:
        open_cells.append([new_cost, cell[0], cell[1]])

In [7]:
def find_min_cost_cell(open_cells):
    min_cell = open_cells[0]
    for cell in open_cells:
        #index 0 of each cell is actually cost
        if(cell[0] < min_cell[0]):
            min_cell = cell
            
    return min_cell

In [8]:
def search(grid,init,goal,cost, delta):
    #list to store the path of cells traversed 
    path = []
    #list of open cells that can be traveresed
    open_cells = []
    
    #create a list to keep track of nodes
    expand = [[-1 for c in range(len(grid[0]))] for r in range(len(grid))]
    
    #variable to keep track of cells visited count
    visited_cells_count = 0

    #current cell to be visited
    current_cell = init
    
    #last visited cell
    last_cell = init
    
    #mark current cell
    grid[current_cell[0]][current_cell[1]] = 1
    
    #place the cell count number to current visited cell
    expand[current_cell[0]][current_cell[1]] = visited_cells_count
    
    #increase cells counter by 1
    visited_cells_count += 1
    
    #add current cell to path as it is traversed 
    path.append([cost, current_cell[0], current_cell[1]])
    
    #find next possible candidate cells that are not occupied and adjacent to current cell
    next_cells = find_available_cells(grid, current_cell, delta)
    #mark available cells as visited
    for cell in next_cells:
        r = cell[0]
        c = cell[1]
        grid[r][c] = 1
        
    #add candidate cells to list of open cells
    add_cells_to_open_list(open_cells, next_cells, cost)
    
    while(len(open_cells) > 0):
        #find a minimum cost cells from the list of cells
        current_cell = find_min_cost_cell(open_cells)
    
        open_cells.remove(current_cell)
        
        #mark current cell
        grid[current_cell[1]][current_cell[2]] = 1
        
        #place the cell count number to current visited cell
        expand[current_cell[1]][current_cell[2]] = visited_cells_count
        #increase cells counter by 1
        visited_cells_count += 1
        
        #add current [cell.cost+1, cell] to path as it is traversed 
        path.append([current_cell[0], current_cell[1], current_cell[2]])
        
        #find next possible candidate cells that are not occupied and adjacent to current cell
        next_cells = find_available_cells(grid, [current_cell[1], current_cell[2]], delta)
        
        #mark available cells as visited
        for cell in next_cells:
            r = cell[0]
            c = cell[1]
            grid[r][c] = 1
        
        #add candidate cells to list of open cells
        add_cells_to_open_list(open_cells, next_cells, current_cell[0] + 1)
        
        last_cell = current_cell
        
        if [last_cell[1], last_cell[2]] == goal:
            return path, expand
    
    return None, expand

In [9]:
import pprint

path, expand = search(grid, init, goal, cost, delta)
if path == None:
    print ('fail')
else:
    print(path[len(path)-1])
    
print(path)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(expand)

# is_move_valid(grid, init, delta[3])
# make_move(grid, init, delta[3])
# find_available_cells(grid, init, delta)

# cells1 = search(grid, init, goal, 4, delta)
# cells = cells + cells1
# cell = find_min_cost_cell(cells)
# print(cell)
# cells.remove(cell)
# print(cells)
# print(cells[0])
# cells.remove(cells[0][0])
# print(cells)
# print(grid)
# print(delta[0])
# print(grid[0][1])


[13, 4, 5]
[[1, 0, 0], [1, 1, 0], [2, 2, 0], [3, 3, 0], [3, 2, 1], [4, 4, 0], [4, 2, 2], [5, 2, 3], [6, 1, 3], [7, 0, 3], [8, 0, 2], [8, 0, 4], [9, 0, 5], [10, 1, 5], [11, 2, 5], [12, 3, 5], [13, 4, 5]]
[   [0, -1, 10, 9, 11, 12],
    [1, -1, -1, 8, -1, 13],
    [2, 4, 6, 7, -1, 14],
    [3, -1, -1, -1, -1, 15],
    [5, -1, -1, -1, -1, 16]]


## Using Heuristic Function to Optimize Search

The heuristic function is a way to inform the search about the direction to a goal. It provides an informed way to guess which neighbor of a node will lead to a goal. There is nothing magical about a heuristic function.

![A*](A_star.png)

As heuristic function gives an estimated guess of distance of current cell from the goal cell so we can use that to improve our search. Now instead of picking cells with minimum cost (also called `g-value`), we are going to pick cells with minimum `f-value` (g-value + h(x, y)) where `h(x,y)` gives the distance from cell (x, y) to goal cell.

In [10]:
#new grid/maze
grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]

#heuristic function filled table which tells a guessed
#distance of goal cell from current cell
#(according to heuristic function show in above image)
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

Now let's modify our search method to use heuristic function to optimize search. 

In [11]:
#function to find possible adjacent cells which are not occupied
def find_available_cells_h(grid, current_cell, moves):
    available_cells = []
    
    for move in moves:
        if (is_move_valid(grid, current_cell, move)):
            available_cells.append(get_next_cell(current_cell, move))
            
    return available_cells

def add_cells_to_open_list_h(open_cells, cells, new_cost):
    for cell in cells:
        #calculate f_value
        f_value = new_cost + heuristic[cell[0]][cell[1]]
        #store the f_value at index 1
        open_cells.append([new_cost, f_value, cell[0], cell[1]])
        
def find_min_cost_cell_h(open_cells):
    min_cell = open_cells[0]
    for cell in open_cells:
        #index 1 of each cell is actually f_value
        #save the cell with minimu f_value
        if(cell[1] < min_cell[1]):
            min_cell = cell
            
    return min_cell

def search_h(grid,init,goal,cost, delta):
    #list to store the path of cells traversed 
    path = []
    #list of open cells that can be traveresed
    open_cells = []
    
    #create a list to keep track of nodes
    expand = [[-1 for c in range(len(grid[0]))] for r in range(len(grid))]
    
    #variable to keep track of cells visited count
    visited_cells_count = 0

    #current cell to be visited
    current_cell = init
    
    #last visited cell
    last_cell = init
    
    #mark current cell
    grid[current_cell[0]][current_cell[1]] = 1
    
    #place the cell count number to current visited cell
    expand[current_cell[0]][current_cell[1]] = visited_cells_count
    
    #increase cells counter by 1
    visited_cells_count += 1
    
    #add current cell to path as it is traversed 
    path.append([cost, current_cell[0], current_cell[1]])
    
    #find next possible candidate cells that are not occupied and adjacent to current cell
    next_cells = find_available_cells_h(grid, current_cell, delta)
    #mark available cells as visited
    for cell in next_cells:
        r = cell[0]
        c = cell[1]
        grid[r][c] = 1
        
    #add candidate cells to list of open cells
    add_cells_to_open_list_h(open_cells, next_cells, cost)
    
    while(len(open_cells) > 0):
        #find a minimum cost cells from the list of cells
        current_cell = find_min_cost_cell_h(open_cells)
        #remove the cell from the list of open cells
        open_cells.remove(current_cell)
        
        g_value = current_cell[0]
        f_value = current_cell[1]
        x = current_cell[2]
        y = current_cell[3]
        
        #mark current cell
        grid[x][y] = 1
        
        #place the cell count number to current visited cell
        expand[x][y] = visited_cells_count
        #increase cells counter by 1
        visited_cells_count += 1
        
        #add current [cell.cost+1, cell] to path as it is traversed 
        path.append([g_value, f_value, x, y])
        
        #find next possible candidate cells that are not occupied and adjacent to current cell
        next_cells = find_available_cells_h(grid, [x, y], delta)
        
        #mark available cells as visited
        for cell in next_cells:
            r = cell[0]
            c = cell[1]
            grid[r][c] = 1
        
        #add candidate cells to list of open cells
        add_cells_to_open_list_h(open_cells, next_cells, g_value+1)
        
        last_cell = current_cell
        
        if [x, y] == goal:
            return path, expand
    
    return None, expand

In [12]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

path, expand = search_h(grid, init, goal, cost, delta)
if path == None:
    print ('fail')
else:
    print(path[len(path)-1])
    
print("Optimal Path [g_value, f_value, x, y]")
pp.pprint(path)
print()
print()
print("Expanded nodes/cells")
pp.pprint(expand)

[9, 9, 4, 5]
Optimal Path [g_value, f_value, x, y]
[   [1, 0, 0],
    [1, 9, 1, 0],
    [2, 9, 2, 0],
    [3, 9, 3, 0],
    [4, 9, 4, 0],
    [5, 9, 4, 1],
    [6, 9, 4, 2],
    [7, 9, 4, 3],
    [8, 9, 4, 4],
    [9, 9, 4, 5]]


Expanded nodes/cells
[   [0, -1, -1, -1, -1, -1],
    [1, -1, -1, -1, -1, -1],
    [2, -1, -1, -1, -1, -1],
    [3, -1, -1, -1, -1, -1],
    [4, 5, 6, 7, 8, 9]]


## Dynamic Programming

`A*` only gives you optimal solution from a given cell to the goal cell but `Dynamic Programming` gives you optimal path to goal cell from any source cell. 

![Dynamic Programming](dynamic-programming/dp.png)

- **Policy** tells you an optimal action (go up, down, left, right on maze) to perform given a cell. Below is a visualization of that taken. 

![Policy](dynamic-programming/policy.png)

- **Value Function**, for each cell, tells you the length of the shortest path to the goal from that cell. Below is a visualization of that. 

![Policy](dynamic-programming/value-function.png)

In [13]:
import copy

# Create a function compute_value which returns
# a grid of values. The value of a cell is the minimum
# number of moves required to get from the cell to the goal. 
#
# If a cell is a wall or it is impossible to reach the goal from a cell,
# assign that cell a value of 99.

# grid = [[0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 0, 0, 0, 1, 0]]

# grid = [[0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 1, 0, 0, 0, 0],
#         [0, 0, 0, 0, 1, 0]]

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 0]]

goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

#Function to check if making the passed move is possible and valid
def is_cell_valid(grid, current_cell, move):
    #calculate the next cell
    next_cell = [current_cell[0] + move[0], current_cell[1] + move[1]]
    
    #make sure we are not overshooting the maze/grid boundary
    if next_cell[0] >= len(grid) or next_cell[1] >= len(grid[0]):
        return False
    
    #make sure we are not undershooting the maze/grid boundary
    if next_cell[0] < 0 or next_cell[1] < 0:
        return False
    
    #make sure the cell is not already occupied, either because 
    #it is an obstacle or we traveresed it before
    if grid[next_cell[0]][next_cell[1]] == 1:
        return False
    
    return True

#function to find possible adjacent cells which are not occupied
def get_neighbors(grid, current_cell, moves):
    neighbors = []
    
    for move in moves:
        if (is_cell_valid(grid, current_cell, move)):
            neighbors.append(get_next_cell(current_cell, move))
            
    return neighbors

def compute_value_recursively(grid_copy, value, current_cell):
    x = current_cell[0]
    y = current_cell[1]
    
    grid_copy[x][y] = 1
    
    #handle base case, which is goal cell
    if [x, y] == goal:
        return 0
    
    neighbors = get_neighbors(grid_copy, current_cell, delta)
    
    min_value = 99
    for cell in neighbors:
        x, y = cell[0], cell[1]
        if grid[x][y] == 1:
            continue;
        
        #get the value for this neigbor 
        v = value[x][y]
        
        #if value is -1, that means value has yet to be calculated
        #so let's calculate
        if v == -1:
            v = compute_value_recursively(grid_copy, value, cell)
            value[x][y] = v
            
        if (v < min_value):
            min_value = v
            
    return min_value + 1

def compute_value(grid,goal,cost):
    #initialize the value table, same size as of grid
    value = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    
    #value for goal cell is 0
    #cost from moving from goal cell to goal cell is 0
    value[goal[0]][goal[1]] = 0

    for row in range(len(grid)):
        for col in range(len(grid[0])):
            
            #for obstacle case, assign value 99
            if grid[row][col] == 1:
                value[row][col] = 99
                continue

            #compute value recursively according to value function formula
            value[row][col] = compute_value_recursively(copy.deepcopy(grid), value, [row, col])
            
    return value

#Function to check if making the passed move is possible and valid
def is_cell_in_grid(grid, current_cell, move):
    #calculate the next cell
    next_cell = [current_cell[0] + move[0], current_cell[1] + move[1]]
    
    #make sure we are not overshooting the maze/grid boundary
    if next_cell[0] >= len(grid) or next_cell[1] >= len(grid[0]):
        return False
    
    #make sure we are not undershooting the maze/grid boundary
    if next_cell[0] < 0 or next_cell[1] < 0:
        return False
    
    return True

def compute_policy(grid, goal, cost):
    value = compute_value(grid, goal, cost)
    print("Value function output")
    pp.pprint(value)
    print()
    
    actions = [[' 'for col in range(len(value[0]))] for row in range(len(value))]
    
    for row in range(len(value)):
        for col in range(len(value[0])):
            
            #check for obstacle case, for which value is 99
            if value[row][col] == 99:
                continue;

            action = ' '
            min_value = value[row][col]

            for i in range(len(delta)):
                if is_cell_in_grid(value, [row, col], delta[i]) == False:
                    continue;

                x, y = row + delta[i][0], col + delta[i][1]
                if value[x][y] < min_value:
                    action = delta_name[i]
                    min_value = value[x][y]

            actions[row][col] = action

    return actions

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(grid)
policy = compute_policy(grid, goal, cost)
pp.pprint(policy)
# v = [[99 for col in range(len(grid[0]))] for row in range(len(grid))]
# print(v)

[   [0, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 1, 0]]
Value function output
[   [13, 99, 7, 6, 5, 4],
    [12, 99, 99, 7, 99, 3],
    [11, 10, 9, 8, 99, 2],
    [12, 99, 99, 99, 99, 1],
    [13, 99, 100, 99, 99, 0]]

[   ['v', ' ', '>', '>', '>', 'v'],
    ['v', ' ', ' ', '^', ' ', 'v'],
    ['>', '>', '>', '^', ' ', 'v'],
    ['^', ' ', ' ', ' ', ' ', 'v'],
    ['^', ' ', '^', ' ', ' ', ' ']]
